#### [D1022_WORK_김현우]
- 바나나랑 사과 사진 구분하기


[1] 모듈 로딩 및 데이터 준비 <hr>

In [54]:
# -------------------------------------------------------------------------
# 1-1. 모듈 로딩
# -------------------------------------------------------------------------
import os, cv2

# 전처리
import pandas as pd
import numpy as np

# 시각화
import seaborn as sns
import koreanize_matplotlib
import matplotlib.pyplot as plt

# 머신러닝
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                             GradientBoostingClassifier, VotingClassifier) 
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
# -------------------------------------------------------------------------
# 1-2. 데이터 로딩
# -------------------------------------------------------------------------
FILE_NAME = './data/csv/fruits.csv'
fruitDF = pd.read_csv(FILE_NAME, header=None)

[2] 데이터 확인 및 전처리 <hr>

In [55]:
# --------------------------------------------------------------
# [2-1] 데이터 확인
# --------------------------------------------------------------
display(fruitDF.head())
display(fruitDF[0].unique())

,0,1,2,3,4,5,6,7,8,9,...,14691,14692,14693,14694,14695,14696,14697,14698,14699,14700
0,apple,43,64,49,30,54,44,30,55,46,...,104,32,77,59,31,75,58,41,80,63
1,apple,64,108,135,64,122,152,53,117,148,...,202,102,205,199,112,214,206,114,218,210
2,apple,160,172,196,161,173,197,161,173,197,...,210,182,192,213,183,191,212,189,197,216
3,apple,52,94,67,142,162,144,215,213,199,...,21,15,26,12,40,56,37,33,51,27
4,apple,202,202,202,201,201,201,200,200,200,...,222,222,222,222,222,222,222,222,222,222


array(['apple', 'banana'], dtype=object)

In [56]:
# --------------------------------------------------------------
# [2-2] 컬럼명 설정
# --------------------------------------------------------------
columns = ['fruit']
for i in range(14700):
    columns.append(f'pixel_{i}')

fruitDF.columns = columns

fruitDF.head()

,fruit,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_14690,pixel_14691,pixel_14692,pixel_14693,pixel_14694,pixel_14695,pixel_14696,pixel_14697,pixel_14698,pixel_14699
0,apple,43,64,49,30,54,44,30,55,46,...,104,32,77,59,31,75,58,41,80,63
1,apple,64,108,135,64,122,152,53,117,148,...,202,102,205,199,112,214,206,114,218,210
2,apple,160,172,196,161,173,197,161,173,197,...,210,182,192,213,183,191,212,189,197,216
3,apple,52,94,67,142,162,144,215,213,199,...,21,15,26,12,40,56,37,33,51,27
4,apple,202,202,202,201,201,201,200,200,200,...,222,222,222,222,222,222,222,222,222,222


[3] 피쳐와 타겟 분리 <hr>

In [57]:
# --------------------------------------------------------------
# [3-1] 타겟 설정
# --------------------------------------------------------------
targetSR = fruitDF['fruit']
print(targetSR)
# --------------------------------------------------------------
# [3-2] 피쳐 설정
# --------------------------------------------------------------
featureDF = fruitDF.iloc[:, 1:]

print(f"\n최종 데이터 shape: featureDF{featureDF.shape}, targetSR{targetSR.shape}")
print(featureDF)


0       apple
1       apple
2       apple
3       apple
4       apple
        ...  
111    banana
112    banana
113    banana
114    banana
115    banana
Name: fruit, Length: 116, dtype: object

최종 데이터 shape: featureDF(116, 14700), targetSR(116,)
     pixel_0  pixel_1  pixel_2  pixel_3  pixel_4  pixel_5  pixel_6  pixel_7  \
0         43       64       49       30       54       44       30       55   
1         64      108      135       64      122      152       53      117   
2        160      172      196      161      173      197      161      173   
3         52       94       67      142      162      144      215      213   
4        202      202      202      201      201      201      200      200   
..       ...      ...      ...      ...      ...      ...      ...      ...   
111      255      255      255      255      255      255      255      255   
112      151      152      148      150      151      147      153      154   
113      255      255      255      255   

[4] 훈련용 / 테스트용 분리 <hr>

In [58]:
x_train, x_test, y_train, y_test = train_test_split(
    featureDF,
    targetSR,
    test_size=0.2,
    random_state=42,
    stratify=targetSR
)

print(featureDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Columns: 14700 entries, pixel_0 to pixel_14699
dtypes: int64(14700)
memory usage: 13.0 MB
None


[5] 랜덤 포레스트 모델 <hr>

In [59]:
# 모델 생성
rfModel = RandomForestClassifier(
    n_estimators=100, 
    random_state=42, 
    n_jobs=-1)

# 학습
rfModel.fit(x_train, y_train)

# 평가
score = rfModel.score(x_test, y_test)
print(f"정확도: {score:.4f}")  

정확도: 0.8333


In [60]:
base_model = RandomForestClassifier(random_state=42, n_jobs=-1)

params = {
    'n_estimators': [50, 100, 200],      # 트리 개수
    'max_depth': [5, 10, 15, None],      # 트리 깊이
    'min_samples_split': [2, 5, 10],     # 노드 분할 샘플
    'min_samples_leaf': [1, 2]
}


grid_search = GridSearchCV(
    base_model, 
    params, 
    cv=5, 
    n_jobs=-1,
    refit=True
)

grid_search.fit(x_train, y_train)

# 결과 
print(f"\n최적 파라미터: {grid_search.best_params_}")
print(f"교차검증 점수: {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_
test_score = best_model.score(x_test, y_test)
print(f"테스트 정확도: {test_score:.4f}")  


최적 파라미터: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
교차검증 점수: 0.8146
테스트 정확도: 0.8333


In [61]:
rf_best = grid_search.best_estimator_

y_pred_rf = rf_best.predict(x_test)

print(" RandomForest- Test Confusion Matrix ")
print(confusion_matrix(y_test, y_pred_rf))

print("\n RandomForest - Test Classification Report ")
print(classification_report(y_test, y_pred_rf))

 RandomForest- Test Confusion Matrix 
[[10  2]
 [ 2 10]]

 RandomForest - Test Classification Report 
              precision    recall  f1-score   support

       apple       0.83      0.83      0.83        12
      banana       0.83      0.83      0.83        12

    accuracy                           0.83        24
   macro avg       0.83      0.83      0.83        24
weighted avg       0.83      0.83      0.83        24



In [68]:
import re
IMG_DIR = "./test"  # 여기만 바꾸면 됨

x_list = []
file_list = []

for fname in sorted(os.listdir(IMG_DIR)):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")):
        continue

    path = os.path.join(IMG_DIR, fname)
    img = cv2.imread(path)
    if img is None:
        continue

    # 학습 때 흑백이면 주석 해제
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img = cv2.resize(img, (70, 70), interpolation=cv2.INTER_AREA)

    # 학습 때 정규화했으면 주석 해제
    # img = img.astype(np.float32) / 255.0

    feat = img.reshape(-1)
    x_list.append(feat)
    file_list.append(fname)

X_pred = np.array(x_list)

# 모델 예측
pred = best_model.predict(X_pred)


# ========== [수정된 부분] 정답 추출 로직 ==========
y_true = []
for fname in file_list:
    # 1. 확장자 제거: "apple (1).jpg" -> "apple (1)"
    name_no_ext = os.path.splitext(fname)[0]
    
    # 2. 괄호와 숫자 제거: "apple (1)" -> "apple"
    # 정규식 설명: \s* (공백) + \( (괄호여는거) + \d+ (숫자들) + \) (괄호닫는거)를 찾아서 지움
    label = re.sub(r'\s*\(\d+\)$', '', name_no_ext)
    
    # 혹시 모를 양옆 공백 제거
    label = label.strip()
    
    y_true.append(label)

# =================================================


# 결과 출력
correct_count = 0
print("\n[예측 결과 확인]")
for fname, p, t in zip(file_list, pred, y_true):
    # p(예측)와 t(정답)가 같은지 비교
    if p == t:
        is_correct = "⭕"
        correct_count += 1
    else:
        is_correct = "❌"
        
    print(f"파일: {fname} | 예측: {p} vs 정답: {t} -> {is_correct}")

# 적중률 출력
accuracy = (correct_count / len(file_list)) * 100
print(f"\n최종 적중률: {accuracy:.2f}% ({correct_count}/{len(file_list)})")


[예측 결과 확인]
파일: apple (01).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (02).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (03).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (04).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (05).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (06).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (07).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (08).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (09).jpg | 예측: banana vs 정답: apple -> ❌
파일: apple (10).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (11).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (12).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (13).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (14).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (15).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (16).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (17).jpg | 예측: banana vs 정답: apple -> ❌
파일: apple (18).jpg | 예측: banana vs 정답: apple -> ❌
파일: apple (19).jpg | 예측: apple vs 정답: apple -> ⭕
파일: apple (20).jpg | 예측: apple vs 정답: apple -> ⭕
파일: a

c:\Users\khw27\anaconda3\envs\ML_CV311\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
